# Solução do problema do Caixeiro Viajante (TSP)

### Uso de três técnicas diferentes
1. Busca exaustiva
2. Hill Climbing
3. Simulated Annealing

In [184]:
# Stephen Marsland, 2008, 2014
# Adaptação e correção de bugs por Hendrik Macedo

import numpy as np
import time

def makeTSP(nCities):
	positions = 2*np.random.rand(nCities,2)-1;
	distances = np.zeros((nCities,nCities))

	for i in range(nCities):
		for j in range(i+1,nCities):
			distances[i,j] = np.sqrt((positions[i,0] - positions[j,0])**2 + (positions[i,1] - positions[j,1])**2);
			distances[j,i] = distances[i,j];

	return distances

def permutation(order):
	order = tuple(order)
	if len(order)==1:
		yield order
	else:
		for i in range(len(order)):
			rest = order[:i] + order[i+1:]
			move = (order[i],)
			for smaller in permutation(rest):
				yield move + smaller

def parameters(nCities):
#	import time
#	nCities = ncities
	distances = makeTSP(nCities)
	return distances

In [185]:
def exhaustive(distances):
	nCities = np.shape(distances)[0]

	cityOrder = np.arange(nCities)

	distanceTravelled = 0
	for i in range(nCities-1):
		distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
	distanceTravelled += distances[cityOrder[nCities-1],0]

	for newOrder in permutation(range(nCities)):
		possibleDistanceTravelled = 0
		for i in range(nCities-1):
			possibleDistanceTravelled += distances[newOrder[i],newOrder[i+1]]
		possibleDistanceTravelled += distances[newOrder[nCities-1],0]
			 
		if possibleDistanceTravelled < distanceTravelled:
			distanceTravelled = possibleDistanceTravelled
			cityOrder = newOrder

	return cityOrder, distanceTravelled

In [206]:
def hillClimbing(distances):

  nCities = np.shape(distances)[0]

  cityOrder = np.arange(nCities)
  np.random.shuffle(cityOrder)

  distanceTravelled = 0
  for i in range(nCities-1):
    distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
  distanceTravelled += distances[cityOrder[nCities-1],0]

  while True:

    neighbors = []
    for i in range(nCities-1):
      for j in range(nCities):
        neighbor = cityOrder.copy()
        neighbor[i] = cityOrder[j]
        neighbor[j] = cityOrder[i]
        neighbors.append(neighbor)

    neighbors = np.array(neighbors)

    best = []
    for i in range(len(neighbors)):
      newDistanceTravelled = 0
      for j in range(nCities-1):
        newDistanceTravelled += distances[neighbors[i][j],neighbors[i][j+1]]
      newDistanceTravelled += distances[cityOrder[nCities-1],0]
      if newDistanceTravelled < distanceTravelled:
        best = neighbors[i]
        distanceTravelled = newDistanceTravelled
          
    if len(best) == 0:   
      break

    cityOrder = best

  return cityOrder, distanceTravelled

In [187]:
def simulatedAnnealing(distances):

  nCities = np.shape(distances)[0]

  cityOrder = np.arange(nCities)
  np.random.shuffle(cityOrder)

  distanceTravelled = 0
  for i in range(nCities-1):
    distanceTravelled += distances[cityOrder[i],cityOrder[i+1]]
  distanceTravelled += distances[cityOrder[nCities-1],0]

  T = 500
  c = 0.8
  time = 1000

	#while T>1:
  for i in range(time):
    if T < 1:
      return cityOrder, distanceTravelled
    # Choose cities to swap
    city1 = np.random.randint(nCities)
    city2 = np.random.randint(nCities)

    if city1 != city2:
      # Reorder the set of cities
      possibleCityOrder = cityOrder.copy()
      possibleCityOrder = np.where(possibleCityOrder==city1,-1,possibleCityOrder)
      possibleCityOrder = np.where(possibleCityOrder==city2,city1,possibleCityOrder)
      possibleCityOrder = np.where(possibleCityOrder==-1,city2,possibleCityOrder)

      # Work out the new distances
      # This can be done more efficiently
      newDistanceTravelled = 0
      for j in range(nCities-1):
        newDistanceTravelled += distances[possibleCityOrder[j],possibleCityOrder[j+1]]
      newDistanceTravelled += distances[cityOrder[nCities-1],0]   
      if (newDistanceTravelled < distanceTravelled) or ((distanceTravelled - newDistanceTravelled) > T*np.log(np.random.rand())):
        distanceTravelled = newDistanceTravelled
        cityOrder = possibleCityOrder

      # Annealing schedule
      T = c*T

  return cityOrder, distanceTravelled

In [188]:
def runExaustive(distances):
	print ("Busca exaustiva")
	start = time.time()
	result = exhaustive(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

def runHill(distances):
	print ("\nHill Climbing")
	start = time.time()
	result = hillClimbing(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

def runSimAnnealing(distances):
	print ("\nSimulated Annealing")
	start = time.time()
	result = simulatedAnnealing(distances)
	finish = time.time()
	print ("Ordem:",result[0]," Distancia:",result[1])
	print ("Tempo:",finish-start)

#runAll()

In [200]:
distances = parameters(5)

In [208]:
runExaustive(distances)

Busca exaustiva
Ordem: (3, 2, 1, 4, 0)  Distancia: 3.1427984024999684
Tempo: 0.0007443428039550781


In [211]:
runHill(distances)


Hill Climbing
Ordem: [3 2 1 4 0]  Distancia: 3.1427984024999684
Tempo: 0.003197193145751953


In [219]:
runSimAnnealing(distances)


Simulated Annealing
Ordem: [3 2 4 1 0]  Distancia: 3.177712659543304
Tempo: 0.0012714862823486328
